# Akıllı Tez Rehberi – Veri Hazırlama (PDF → Parça Metin)

Bu not defteri, lisans tezimizi (PDF dosyası: `yapayzekadilmodelleri.pdf`) bir RAG (Retrieval-Augmented Generation) tabanlı sohbet asistanında kullanılmaya uygun hale getirir.

Bu kodda sırayla şunları yapacağız:

1) PDF dosyasını not defterine yüklediğimizi doğrulayacağız.
   
2) PDF’ten sayfa sayfa metin çıkaracağız.
 
3) Metni sadeleştireceğiz (gereksiz boşlukları/karakterleri temizleyeceğiz).
   
4) Uzun metni “küçük parçalara” (chunk) böleceğiz.
 
5) Her parçaya faydalı “etiketler (metadata)” ekleyeceğiz (ör. sayfa numarası, kaynak adı).
 
6)  Sonuçları iki dosya biçiminde kaydedeceğiz: JSONL (RAG ingest için çok pratik) ve Parquet (tablo gibi hızlı).

Neden böyle?
- RAG sistemi, uzun metni küçük parçalara ayırdığımızda çok daha iyi arama yapar.
- Soru geldiğinde “en ilgili parçalar” bulunur ve yanıt bunlara dayanır.
- Metadata, cevabın “hangi sayfadan geldiğini” göstermemize imkân verir.

Önemli:
- Bu defter sadece “veri hazırlama” içindir.
- Web arayüzü ve RAG akışı `project.py` içinde (Hugging Face Spaces üzerinde) çalışacaktır.

In [1]:
# Amaç:
# 1) "Temiz çıktı": Kullanıcıyı boğacak uyarıları ve logları kısmak.
# 2) Gerekli paketleri (pypdf, pandas) sessizce kurmak.
#
# Neden önemli?
# - Notebook çıktıları zamanla çok kalabalıklaşabilir. Odak kaybını önlemek için
#   yalnızca anlamlı, kısa onay mesajları basacağız.
# - pypdf: PDF içinden metin çekmek için gereklidir.
# - pandas: parçaları tabloya dökmek ve dosyaya kaydetmek için kullanacağız.

import warnings, logging
warnings.filterwarnings("ignore")                  # Örn. Future/Deprecation uyarılarını gösterme
logging.getLogger().setLevel(logging.ERROR)        # Genel log seviyesini ERROR'a indir

!pip -q install pypdf pandas

print("✅ Ortam hazır: Uyarılar kısıldı, pypdf & pandas kuruldu.")

✅ Ortam hazır: Uyarılar kısıldı, pypdf & pandas kuruldu.


## 1) PDF dosyasını bulma (otomatik yol tespiti) ve doğrulama

Kaggle’da dosyalar iki tip konumda bulunabilir:
- Çalışma alanı yüklemeleri: `/kaggle/working/…` (Files → Upload)
- Dataset ekleri: `/kaggle/input/<dataset-adı>/…` (Notebook → Add data)

Neden otomatik arama?
- Kullanıcı bazen “Upload” ile, bazen “Add data” ile ekler. Tek bir sabit yol hata üretir.
- Kod, `yapayzekadilmodelleri.pdf` adını tüm olası konumlarda arayıp ilk bulduğu geçerli yolu `PDF_PATH` olarak ayarlar.
- Bulunamazsa, kullanıcıyı doğru adımlara yönlendiren net bir hata mesajı verir.

In [2]:
# Bu hücre, 'yapayzekadilmodelleri.pdf' dosyasını otomatik olarak bulur.
# Adımlar:
# 1) En olası konumlar listelenir (çalışma dizini, /kaggle/working, /kaggle/input).
# 2) /kaggle/input altında bağlanan tüm dataset klasörleri de taranır.
# 3) İlk bulunan geçerli yol seçilir. Bulunamazsa, recursive arama yapılır.
# 4) Son olarak assert ile kesin doğrulama yapılır; bulunamadıysa yönlendirici hata verilir.

import os, glob

PDF_NAME = "yapayzekadilmodelleri.pdf"  # TEZ PDF dosya adınız (uzantı dahil, harf uyumuna dikkat)

# 1) En olası sabit konum adayları
candidates = [
    f"./{PDF_NAME}",                    # Mevcut çalışma dizini
    f"/kaggle/working/{PDF_NAME}",     # Files → Upload sonrası tipik konum
    f"/kaggle/input/{PDF_NAME}",       # Nadir: input kökünde olabilir
]

# 2) /kaggle/input altındaki dataset klasörlerini dolaş ve olası yolu ekle
for d in glob.glob("/kaggle/input/*"):
    candidates.append(os.path.join(d, PDF_NAME))

# 3) İlk var olan yolu seç
PDF_PATH = next((p for p in candidates if os.path.exists(p)), None)

# 4) Hâlâ bulunamadıysa, tüm input altında derin arama yap
if PDF_PATH is None:
    matches = glob.glob(f"/kaggle/input/**/{PDF_NAME}", recursive=True)
    PDF_PATH = matches[0] if matches else None

# 5) Son doğrulama ve yönlendirme
assert PDF_PATH and os.path.exists(PDF_PATH), (
    "PDF bulunamadı. Notebook'a 'Add data' ile ilgili dataset'i ekleyin veya Files→Upload ile PDF'yi yükleyin. "
    "Dosya adı tam olarak 'yapayzekadilmodelleri.pdf' olmalı."
)

print(f"✅ PDF bulundu: {PDF_PATH}")

✅ PDF bulundu: /kaggle/input/yapayzekadilmodelleri/yapayzekadilmodelleri.pdf


## 2) Metni biçimsel olarak temizleme (neden?)

PDF’ten çekilen ham metinde:
- Bozuk/gizli karakterler (ör. `\x00`)
- Birden fazla boşluk veya tab
- Birbiri ardına aşırı sayıda satır sonu olabilir.

Bunlar “anlamı” değiştirmeden temizlenir:
- Bozuk karakterler boşluğa çevrilir.
- Çoklu boşluk/tab tek boşluğa indirilir.
- 3+ satır sonu 2 satıra düşürülür.

Amaç:
- Okunabilirlik ve arama isabetini artırmak (parçalama adımı daha tutarlı çalışır).

In [3]:
# Bu fonksiyon ham metni biçimsel olarak sadeleştirir; semantik içerik korunur.
# Neden bu kadar basit? Aşırı agresif temizlik (ör. stopword silme) içerik kaybına yol açabilir;
# burada yalnızca "biçimsel" normalizasyon hedeflenir.

import re

def clean_text(text: str) -> str:
    # 1) Bozuk null karakterlerini boşlukla değiştir (çoğu zaman render hatasıdır)
    text = text.replace("\x00", " ")
    # 2) Birden fazla boşluk veya tab'ı tek boşluğa indir
    text = re.sub(r"[ \t]+", " ", text)
    # 3) 3 veya daha fazla ardışık satır sonunu 2 satıra düşür (okunabilirlik için)
    text = re.sub(r"\n{3,}", "\n\n", text)
    # 4) Baştaki/sondaki fazlalıkları temizle
    return text.strip()

print("✅ Temizleme fonksiyonu hazır.")

✅ Temizleme fonksiyonu hazır.


## 3) PDF’ten sayfa sayfa metin çıkarma

Adımlar:
1) PDF baytları belleğe alınır (dosyayı her seferinde diskten okumamak için).
2) `pypdf.PdfReader` ile her sayfanın metni `extract_text()` ile alınır.
3) Metin `clean_text()` ile biçimsel olarak sadeleştirilir.
4) Her sayfanın sonucu `pages_text` listesine eklenir (liste elemanı = sayfa metni).

Not:
- Bazı sayfalar tarama (görsel) olabilir; metin boş dönebilir. MVP için yeterli; gerekirse OCR eklenebilir.

In [4]:
# PDF'i bellekten okuyup tüm sayfalardan metni çıkarır ve temizler.
# Neden bellekten? I/O (girdi/çıktı) sayısını azaltmak, performansı artırmak.

import io
from typing import List
from pypdf import PdfReader

# 1) PDF'i bayt olarak belleğe al
with open(PDF_PATH, "rb") as f:
    pdf_bytes = f.read()

# 2) Baytları PdfReader'a ver
reader = PdfReader(io.BytesIO(pdf_bytes))

# 3) Toplam sayfa sayısını al
num_pages = len(reader.pages)

# 4) Her sayfanın metnini al, temizle ve listeye ekle
pages_text: List[str] = []
for i in range(num_pages):
    try:
        raw = reader.pages[i].extract_text() or ""   # Metin yoksa boş string
    except Exception:
        raw = ""                                     # Okunamayan sayfa varsa boş kabul
    pages_text.append(clean_text(raw))               # Biçimsel temizlikten geçir

# 5) Kısa özet metrik (temiz çıktı)
total_chars = sum(len(t) for t in pages_text)
print(f"✅ Metin çıkarıldı: sayfa={num_pages}, toplam_karakter={total_chars:,}")

✅ Metin çıkarıldı: sayfa=126, toplam_karakter=279,069


# 4) Metni parçalara ayırma (chunking)

# Neden?

 -RAG, uzun metni küçük parçalara ayrılmış halde çok daha isabetli tarar.
 
# Nasıl?

-Yaklaşık 1500 kelimelik parçalar üretilir (ARTIRILDI: 512'den 1500'e).

-Parçalar arası 250 kelimelik "örtüşme (overlap)" bırakılır (ARTIRILDI: 80'den 250'e):

-Paragraph birden fazla parçaya bölünse bile bağlam korunur.

# Neden bu değerler?

- CHUNK_SIZE=512 çok küçüktü → aynı sayfadaki bilgiler parçalanıyordu

- CHUNK_SIZE=1500 → daha geniş bağlam, aynı chunk'ta ilgili bilgiler

- CHUNK_OVERLAP=250 → daha fazla örtüşme = daha yumuşak geçişler, bağlam kaybı yok

In [5]:


from typing import Dict

CHUNK_SIZE = 1500      # DÜZELTME: 512'den 1500'e arttırıldı (daha büyük bağlam)
CHUNK_OVERLAP = 250    # DÜZELTME: 80'den 250'e arttırıldı (daha fazla örtüşme)

records: List[Dict] = []                 # Nihai kayıtlar (content + meta) burada toplanacak
title = "Yapay Zekâ Dil Modelleri – Lisans Tezi"
source = os.path.basename(PDF_PATH)      # Örn. 'yapayzekadilmodelleri.pdf'

for page_idx, text in enumerate(pages_text, start=1):
    words = text.split()                 # Sayfadaki metni kelimelere böl (basit ve hızlı)
    if not words:                        # Sayfa tamamen boşsa (görsel vs.), atla
        continue
    start = 0                            # Kaygan pencere (sliding window) başlangıcı
    while start < len(words):
        end = start + CHUNK_SIZE         # Pencere bitişi (1500 kelime sonrası)
        chunk_words = words[start:end]
        chunk_text = " ".join(chunk_words).strip()
        if chunk_text:
            # Parça + minimal metadata: başlık, kaynak dosya, sayfa aralığı
            records.append({
                "content": chunk_text,
                "title": title,
                "source": source,
                "page_start": page_idx,  # Basit yaklaşım: parça tek sayfadan
                "page_end": page_idx
            })
        # Örtüşme: bir sonraki parçaya geçerken 250 kelime geri sar
        next_start = end - CHUNK_OVERLAP
        start = next_start if next_start > 0 else end

print(f"✅ Parçalama tamam: toplam_parça={len(records)}")
print(f"   Chunk boyutu: {CHUNK_SIZE} kelime")
print(f"   Örtüşme: {CHUNK_OVERLAP} kelime")

✅ Parçalama tamam: toplam_parça=126
   Chunk boyutu: 1500 kelime
   Örtüşme: 250 kelime


## 5) Önizleme – Parçalar (önerilen) ve Ham Sayfalar (opsiyonel)

Amaç:
- Parça (chunk) önizlemesi ile RAG’in kullanacağı veriyi hızlıca gözden geçirmek.
- Gerekirse ham sayfa metni önizlemesi ile metin çıkarma kalitesini kontrol etmek.

Neden iki tür önizleme?
- Parça önizleme: RAG doğrudan bu parçalarla çalışır; içerik kesiti ve metadata (başlık, kaynak, sayfa) bir arada görülür. Varsayılan ve önerilen yöntemdir.
- Ham sayfa önizleme: Metin çıkarma aşamasında boş/gürültülü sayfaları tespit etmek, OCR ihtiyacını görmek için yararlıdır. Gerektiğinde açılır.

Kullanım:
- `PREVIEW_N`: Kaç parça gösterileceğini belirler (ör. 10, 13, 20).
- `PREVIEW_CHARS`: Her parça içeriğinin kaç karakterde kesileceğini belirler (örn. 200–300). Çıktıyı kalabalıklaştırmamak için kısaltılır.
- `PAGE_PREVIEW`: `True` yapıldığında ham sayfa önizlemesi de çalışır.
- `PAGE_PREVIEW_N`: Kaç sayfa gösterileceğini belirler (örn. 3, 10, 20).
- `PAGE_PREVIEW_CHARS`: Her sayfa metninin kaç karakterde kesileceğini belirler (örn. 800).

Beklenen çıktı:
- Parça önizlemesi: Küçük bir tablo; `content_preview` sütununda metnin kesiti ve `title/source/page_start/page_end` metadata’sı.
- Ham sayfa önizlemesi (opsiyonel): Konsola “— Sayfa i —” başlıkları ile kısa metin kesitleri.

Not:
- Parça önizleme RAG kalitesi için ana referanstır. Ham sayfa önizleme yalnızca teşhis amaçlıdır.
- Eğer “Önizleme yok” uyarısı görürseniz, ilgili önceki adımı (parçalama veya metin çıkarma) çalıştırın.

In [6]:
# ——— ÖNİZLEME AYARLARI ———
PREVIEW_N = 13          # Kaç kayıt gösterilecek? (örn. 10, 13, 20)
PREVIEW_CHARS = 220     # İçerik kaç karakterde kesilsin? (örn. 200–300)

# ——— CHUNK ÖNİZLEME (ÖNERİLEN) ———
# RAG bu parçalarla çalıştığı için kaliteyi burada kontrol etmek en mantıklısıdır.
import pandas as pd

if 'records' in globals() and isinstance(records, list) and len(records) > 0:
    preview = pd.DataFrame(records[:PREVIEW_N]).copy()
    # İçeriği keserek göster (çıktıyı temiz ve okunur tutmak için)
    preview["content_preview"] = preview["content"].str.slice(0, PREVIEW_CHARS) + "..."
    display(preview[["content_preview", "title", "source", "page_start", "page_end"]])
else:
    print("Önizleme yok: 'records' boş ya da tanımsız. Parçalama adımını çalıştırdığınızdan emin olun.")

# ——— (OPSİYONEL) HAM SAYFA ÖNİZLEME ———
# Metin çıkarma/boş sayfa/OCR ihtiyacını teşhis etmek için gerektiğinde açın.
PAGE_PREVIEW = True    # True yapınca ham sayfa önizlemesini de görürsün
PAGE_PREVIEW_N = 3    # Kaç sayfa gösterilecek? (örn. 10 veya 20)
PAGE_PREVIEW_CHARS = 800

if PAGE_PREVIEW:
    if 'pages_text' in globals() and isinstance(pages_text, list) and len(pages_text) > 0:
        for i, t in enumerate(pages_text[:PAGE_PREVIEW_N], start=1):
            print(f"--- Sayfa {i} ---")
            print((t[:PAGE_PREVIEW_CHARS] + "...") if len(t) > PAGE_PREVIEW_CHARS else t)
            print()
    else:
        print("Ham sayfa önizleme için 'pages_text' bulunamadı. Metin çıkarma adımını çalıştırın.")

,content_preview,title,source,page_start,page_end
0,T.C. EGE ÜNİVERSİTESİ Fen Fakültesi Matematik ...,Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,1,1
1,T.C. EGE ÜNİVERSİTESİ Fen Fakültesi YAPAY ZEKÂ...,Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,2,2
2,"iii ÖZET YAPAY ZEKÂ DİL MODELLERİ ÇORUM, Yağmu...",Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,3,3
3,iv ABSTRACT ARTIFICIAL INTELLIGENCE LANGUAGE M...,Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,4,4
4,v İÇİNDEKİLER Sayfa İÇ KAPAK ....................,Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,5,5
5,vi 3.2.1 Yapay Sinir Ağları ( Artificial Neura...,Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,6,6
6,vii 6.1.1 Çok Modlu Büyük Dil Modelleri (Multi...,Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,7,7
7,viii ŞEKİLLER DİZİNİ Şekil Sayfa Şekil 2.1 Tur...,Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,8,8
8,ix ŞEKİLLER DİZİNİ (devam) Şekil Sayfa Şekil 5...,Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,9,9
9,x TABLOLAR DİZİNİ Tablo Sayfa Tablo 4.1 Dil mo...,Yapay Zekâ Dil Modelleri – Lisans Tezi,yapayzekadilmodelleri.pdf,10,10


--- Sayfa 1 ---
T.C. 
EGE ÜNİVERSİTESİ 
Fen Fakültesi 
Matematik Anabilim Dalı Adı 
 
İzmir 
2025 
 
Yağmur ÇORUM 
 
YAPAY ZEKÂ DİL MODELLERİ 
Lisans Tezi

--- Sayfa 2 ---
T.C. 
EGE ÜNİVERSİTESİ 
Fen Fakültesi 
 
 
YAPAY ZEKÂ DİL MODELLERİ 
 
 
Yağmur ÇORUM 
Danışman: Prof. Dr. Burak ORDİN 
 
 
İzmir 
2025 
Matematik Anabilim Dalı Adı

--- Sayfa 3 ---
iii 
 
 
 
ÖZET 
YAPAY ZEKÂ DİL MODELLERİ 
ÇORUM, Yağmur 
Lisans Tezi, Matematik Anabilim Dalı 
 
Tez Danışmanı: Prof. Dr. Burak ORDİN 
 
Temmuz 2025, 114 sayfa 
Bu tezde, yapay zekâ ve doğal dil işleme alanındaki güncel gelişmeler ve dil 
modelleme yöntemleri kapsamlı bir şekilde incelenmektedir. Literatür taraması yöntemi 
kullanılarak, yapay zekânın tanımı, tarihçesi ve doğal dil işlemenin evrimi sistemli bir 
şekilde ele alınmıştır. Makine öğrenimi ve derin öğrenmenin temel prensipleri, denetimli, 
denetimsiz ve takviyeli öğrenme yaklaşımları açıklanmıştır. Dil modellerinin başarısında 
önemli rol oynayan yapay sinir ağları ve derin ö

## 6) Çıktıları kaydetme (JSONL ve Parquet)

Bu adımda, hazırladığımız metin parçalarını iki farklı dosya biçiminde kaydediyoruz. Sonraki “ingest” (sisteme alma) aşamasında bu dosyalar kullanılacak.

Neden iki biçim?

- JSONL (JSON Lines)
  - Yapısı: Her satır bir kayıt içerir → { "content": "...", "meta": { ... } }
  - Ne zaman kullanılır? RAG’e veri “aktarırken” satır satır okumak çok pratiktir.
  - Avantaj: Büyük dosyalarda akış halinde işlemek kolaydır; çoğu ingest aracı JSONL’yi sever.

- Parquet
  - Yapısı: Tablo (sütunlu) format → hızlı okunur, kolay filtrelenir.
  - Ne zaman kullanılır? Elle kalite kontrol, basit analiz ve hızlı inceleme için idealdir.
  - Avantaj: Aynı veriyi genelde daha küçük boyutta saklar; performanslıdır.

Kısaca:
- JSONL: RAG’e veri aktar (ingest) ederken tercih edilen format.
- Parquet: İnsan gözüyle kontrol/analiz yaparken rahat format.

Bu dosyalar, web uygulamasında (`project.py`) indeksleme ve arama için kaynak olarak kullanılacaktır.

In [7]:
# Çıktı dosyalarını üretir:
# - processed_docs.jsonl: satır başına 'content' + 'meta' JSON objesi
# - processed_docs.parquet: tablo formatında tüm parçalar
# Not: JSONL, satır satır okunabildiği için akış tabanlı işlemlerde çok pratiktir.

import json

jsonl_path = "processed_docs.jsonl"
parquet_path = "processed_docs.parquet"

# 1) JSONL yazımı: her satırda 'content' ve 'meta' alanları olan bir JSON objesi bulunur
with open(jsonl_path, "w", encoding="utf-8") as f:
    for r in records:
        row = {
            "content": r["content"],
            "meta": {
                "title": r["title"],
                "source": r["source"],
                "page_start": r["page_start"],
                "page_end": r["page_end"]
            }
        }
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

# 2) Parquet yazımı: tüm kayıtları tek seferde tabloya döker
pd.DataFrame(records).to_parquet(parquet_path, index=False)

print(f"✅ Kaydedildi: {jsonl_path}, {parquet_path}")

✅ Kaydedildi: processed_docs.jsonl, processed_docs.parquet
